<a href="https://colab.research.google.com/github/meg190/aplicadas-II-colabs/blob/main/aplicadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análisis de transiciones académicas usando cadenas de markov

##Resumen

Comprender la dinámica del rendimiento académico a lo largo del semestre es clave para realizar mejoras pedagógicas y anticipar riesgos en el aprendizaje. Por ello, en este trabajo se modelaron las transiciones entre cuatro niveles de desempeño estudiantil (Básico, En desarrollo, Alto y Sobresaliente) utilizando una cadena de Markov de tiempo discreto. Se construyó la matriz de transición entre evaluaciones, se estimó la distribución estacionaria del sistema y se calcularon probabilidades asociadas a cambios relevantes en el desempeño.Los resultados mostraron que con el paso de las evaluaciones el sistema converge hacia una distribución dominada por niveles medios y altos, y que las probabilidades de descender a estados críticos permanecen bajas. Estas tendencias indican un escenario académico con progresión gradual y estabilidad en el desempeño estudiantil.

##Objetivos
* Estimar la matriz de transición entre niveles académicos a lo largo de las evaluaciones.
* Obtener la distribución estacionaria del sistema.
Calcular probabilidades específicas:  i.  desde Básico → aprobado en 4 evaluaciones ii.  desde Sobresaliente → Básico en 3 evaluaciones.
*Discutir implicaciones pedagógicas y limitaciones.

##Introducción

El rendimiento académico es un indicador fundamental para comprender cómo avanzan los estudiantes a lo largo del tiempo y para orientar intervenciones pedagógicas que fortalezcan el aprendizaje. Desde la psicología educativa, Zimmerman (2002) señala que el desempeño escolar evoluciona dinámicamente conforme los estudiantes se enfrentan a nuevas tareas, regulan su aprendizaje y responden a distintos contextos. Esta perspectiva resalta la necesidad de analizar el rendimiento no como un valor aislado, sino como una trayectoria compuesta por múltiples estados posibles.

Para modelar procesos que evolucionan por etapas y con comportamiento probabilístico, las cadenas de Markov ofrecen un marco teórico adecuado. Feller (1968) describe estos modelos como sistemas estocásticos en los que la transición hacia un estado futuro depende únicamente del estado actual, permitiendo representar fenómenos evolutivos en secuencias discretas. Esta propiedad las convierte en una herramienta útil para estudiar cambios graduales en variables educativas, donde los estudiantes pueden avanzar, mantenerse o retroceder de un estado de rendimiento a otro.

La aplicación de cadenas de Markov al ámbito educativo ha mostrado resultados prometedores en la caracterización de rutas de aprendizaje. En un estudio realizado con estudiantes universitarios, Brezavšček, Pejić Bach y Baggia (2017) emplearon una cadena de Markov para analizar la evolución del rendimiento estudiantil a lo largo de varios semestres, demostrando que las transiciones entre estados académicos presentan estructuras probabilísticas estables y predecibles. Sus hallazgos evidencian que este tipo de modelos permite estimar riesgos, reconocer trayectorias típicas de avance o estancamiento y aportar información útil para la toma de decisiones institucionales orientadas al mejoramiento del aprendizaje.

Por lo anterior, en este trabajo se modelaron las transiciones entre cuatro niveles de rendimiento académico (Básico, En desarrollo, Alto y Sobresaliente) mediante una cadena de Markov de tiempo discreto. El objetivo fue estimar la matriz de transición, determinar la distribución estacionaria y calcular probabilidades de interés con el fin de interpretar la estabilidad del sistema y comprender la evolución del desempeño estudiantil a lo largo del semestre.

##Antecedentes

El estudio de procesos educativos mediante modelos probabilísticos ha tenido un importante desarrollo en las últimas décadas. Desde la teoría de probabilidad, las cadenas de Markov permiten describir sistemas que evolucionan por etapas, representando transiciones entre estados con base en evidencia empírica. En contextos educativos, esta perspectiva ha sido útil para analizar cómo los estudiantes progresan o retroceden en su rendimiento según condiciones específicas del entorno académico.

Investigaciones orientadas al análisis del desempeño estudiantil han encontrado que los patrones de aprendizaje suelen organizarse en estados relativamente estables. Kuh (2008), en sus estudios sobre compromiso académico y éxito universitario, destaca que los estudiantes tienden a mantener comportamientos consistentes a lo largo del semestre, salvo intervenciones pedagógicas significativas. Esto indica que los modelos basados en transiciones entre estados pueden capturar adecuadamente la dinámica real del rendimiento.

Además, se han desarrollado estudios cuantitativos que aplican cadenas de Markov para comprender la evolución académica en educación superior. Por ejemplo, Olmedo-Alonso (2020) utilizó una cadena de Markov absorbente para analizar la permanencia y el riesgo de abandono, demostrando que los estados académicos críticos suelen actuar como “puntos de atracción” para ciertos perfiles estudiantiles. Esta evidencia sugiere que las matrices de transición pueden emplearse para estimar probabilidades de recuperación o deterioro académico.

Finalmente, los estudios recientes sobre el uso de modelos probabilísticos en educación, como el análisis presentado en Polo del Conocimiento (2023), evidencian que las metodologías basadas en transiciones entre estados permiten comprender mejor la dinámica del rendimiento académico a lo largo del tiempo. En este tipo de investigaciones se muestra que los patrones de avance o retroceso de los estudiantes no ocurren al azar, sino que reflejan comportamientos sistemáticos que pueden analizarse mediante matrices de transición y estimaciones de probabilidad. Este enfoque no solo facilita la descripción del movimiento entre niveles de desempeño, sino que también aporta criterios útiles para anticipar trayectorias futuras y orientar intervenciones pedagógicas dirigidas a los grupos con mayor vulnerabilidad académica.

##Metodología
Para el desarrollo del análisis se siguió un diseño cuantitativo de tipo descriptivo, basado en el estudio de transiciones entre categorías de rendimiento académico mediante cadenas de Markov. El procedimiento se estructuró en tres fases principales:

**1. Recolección y organización de los datos:**

Se recopilaron las calificaciones correspondientes a las evaluaciones realizadas durante un periodo académico establecido con datos genericos. Las notas obtenidas se registraron en una matriz de datos inicial que permitió identificar la trayectoria de cada estudiante a lo largo de las diferentes mediciones.

**2. Clasificación de los estudiantes en estados académicos**

Las calificaciones fueron categorizadas según los cuatro estados académicos definidos en el marco teórico:
* Básico (0.0-2.9)
* En desarrollo (3.0-3.7)
* Alto grado (3.8-4.4)
* Sobresaliente (4.5-5.0)

 Cada estudiante fue asignado a un estado para cada momento de evaluación, generando así una secuencia ordenada de estados.

**3. Construcción de la matriz de transición**

A partir de las secuencias de estados se contabilizaron las transiciones ocurridas entre una evaluación y la siguiente. Posteriormente, se normalizaron estas frecuencias para obtener la matriz de transición de primer orden, donde cada elemento representa la probabilidad de pasar de un estado académico a otro en una evaluación posterior.

**4. Análisis de dinámica y predicción**

Con la matriz de transición construida, se aplicaron los procedimientos propios de las cadenas de Markov para proyectar el comportamiento del sistema en pasos futuros. Esto incluyó el cálculo de potencias de la matriz de transición y la estimación de la distribución estacionaria mediante métodos algebraicos y computacionales.

##Resultados







Lo que tenemos en los datos son las probabilidades de pasar de un estado de bajo, basico, alto y superior a lo largo de un semestre, donde cada momento representa una evaluacion, y lo ideal es calcular cosas respecto a eso. en python está la matriz de transición que da las probabilidades.

In [1]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/Juanm3005/proyecto-aplicadas-II/refs/heads/main/aplicadas%20proyecto.csv"
df = pd.read_csv(url, sep=";", decimal=",")

def nota_a_estado(n):
    if n <= 2.9:
        return 0  # Básico
    elif 3.0 <= n <= 3.7:
        return 1  # En desarrollo
    elif 3.8 <= n <= 4.4:
        return 2  # Alto grado
    elif 4.5 <= n <= 5.0:
        return 3  # Sobresaliente
    else:
        return np.nan  # si hay valores fuera de rango

# 3. Convertir todas las notas en estados
# Suponiendo que las columnas de notas son algo como "momento1", "momento2", …
cols_notas = [c for c in df.columns if c.startswith("momento")]
df_states = df[cols_notas].map(nota_a_estado)

# 4. Construir matriz de conteos N
n_states = 4
N = np.zeros((n_states, n_states), dtype=int)
for idx, row in df_states.iterrows():
    estados = row.dropna().tolist()
    for a, b in zip(estados[:-1], estados[1:]):
        if not np.isnan(a) and not np.isnan(b):
            N[int(a), int(b)] += 1

# 5. Calcular matriz de transición P (con suavizado, ej. α=1)
alpha = 1.0
Np = N + alpha
row_sums = Np.sum(axis=1, keepdims=True)
P = Np / row_sums

# 6. Mostrar resultados
print("Matriz de conteos N:")
print(N)
print("\nMatriz de transición P:")
print(np.round(P.T, 4))
print("\nSuma de columnas de P:", np.round(P.sum(axis=1), 4))

Matriz de conteos N:
[[32  6  3 13]
 [ 6  3  5 16]
 [ 3  5 12 24]
 [16 15 35 96]]

Matriz de transición P:
[[0.569  0.2059 0.0833 0.1024]
 [0.1207 0.1176 0.125  0.0964]
 [0.069  0.1765 0.2708 0.2169]
 [0.2414 0.5    0.5208 0.5843]]

Suma de columnas de P: [1. 1. 1. 1.]


##Analisis de resultados

Preguntas a resolver:

*   ¿Cómo se mueven los estudiantes entre niveles académicos?
*   ¿Cuál es el estado de equilibrio del sistema? (¿Las notas se mantienen estables?)
* ¿Cuál es la probabilidad de que un estudiante que empieza perdiendo pueda ganar la materia si hay 4 evaluaciones?
* ¿Cuál es la probabilidad de que un estudiante que empezó sobresaliente esté en riesgo de perder después de 3 evaluaciones?

##Conclusiones


##Bibliografía

1. Zimmerman, B. (2002). Becoming a self-regulated learner: An overview. Theory Into Practice tomado de: https://www.leiderschapsdomeinen.nl/wp-content/uploads/2016/12/Zimmerman-B.-2002-Becoming-Self-Regulated-Learner.pdf

2. Feller, W. (1968). An Introduction to Probability Theory and Its Applications. Wiley. paginas 444-446. tomado de https://bitcoinwords.github.io/assets/papers/an-introduction-to-probability-theory-and-its-applications.pdf

3. Markov Analysis of Students’ Performance and Academic Progress in Higher Education de Brezavšček, Pejić Bach & Baggia (2017). Tomado de:https://reference-global.com/2/v2/download/pdf/10.1515/orga-2017-0006

4. Kuh, G. (2008). High-Impact Educational Practices. AAC&U. Tomado de: https://navigate.utah.edu/_resources/documents/hips-kuh-2008.pdf

5. Olmedo-Alonso, J. (2020). Cadenas de Markov absorbentes para analizar el abandono escolar. Revista ICBI.Tomado de: https://repository.uaeh.edu.mx/revistas/index.php/icbi/article/view/13810/12272

6. Colcha Aucancela, C. A., Colcha Aucancela, L. del R., Hinojosa Secaira, F. G., Urquizo Hinojosa, E. I., Litardo Arias, T. E. (2025). “Modelado predictivo del rendimiento académico y detección de trayectorias de riesgo mediante cadenas de Markov e IA diseño de estrategias adaptativas en educación personalizada utilizando Big Data". Tomado de: https://polodelconocimiento.com/ojs/index.php/es/article/view/10210/pdf
